In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "My First Project-fe78171bdba0.json"
import pandas as pd
from google.cloud import bigquery
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

## Import Jobs Dataframe from Google BigQuery

In [2]:
project_id='deep-rigging-245523'
query = "SELECT * FROM Projects.job_listings"
jobs_df = (pd.read_gbq(query,project_id=project_id, dialect='standard')
           .drop_duplicates(subset = "description"))
print(len(jobs_df))
jobs_df.head()

523


,title,company,location,summary,link,description
0,Staff Data Scientist,Valassis Digital,"Morrisville, NC",Staff Data Scientist – Machine Lea...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Staff Data Scientist – Machine Learning & Real...
1,People Analytics - data scientist,IBM,"Armonk, NY",Formulating mathematical approache...,https://www.indeed.com/rc/clk?jk=f423a0f019acd...,"Introduction At IBM, work is more than a job -..."
2,Data Scientist Analyst,Domino's,"Ann Arbor, MI",Proficient in SQL and R or Python....,https://www.indeed.com/rc/clk?jk=e6f0a2235caab...,"Domino’s, which began in 1960 as a single stor..."
3,Deloitte Global- Data Scientist,Deloitte,"Glen Mills, PA",Train other data scientists in are...,https://www.indeed.com/rc/clk?jk=2f3637eb11fb2...,"Deloitte leads with purpose, solving complex i..."
4,Data Scientist (Anti-Fraud Team),Walmart eCommerce,"San Francisco Bay Area, CA",Deep knowledge of statistical anal...,https://www.indeed.com/company/Walmart-eCommer...,Imagine working in an environment where one ex...


## Import Skills Dataframe from Google BigQuery

In [3]:
query = "SELECT * FROM Projects.skills"
skills_df = pd.read_gbq(query,project_id=project_id, dialect='standard')
skills_df = skills_df.rename({'string_field_0': 'name'}, axis='columns')
print(len(skills_df))
skills_df.head(n=10)

36944


,name
0,(ISC)2
1,.NET
2,.NET CLR
3,.NET Compact Framework
4,.NET Framework
5,.NET Remoting
6,.Net Core
7,.com
8,.htaccess
9,1-4 Units


## User Inputs

In [4]:
input_profession = input("Enter your profession, or enter x to use default inputs ")
if input_profession == "x":
    input_profession = "Data Analyst"
    input_skills = ["Python", "R", "SQL", "Tableau", "Power BI", "Salesforce", "Excel",
                             "Machine Learning", "Statistics", "Analytics", "Visualization", "Web Scraping"]
    print("Default profession: ", input_profession)
    print("Default skills: ", input_skills)
else: 
    amount_skills = int(input("How many skills do you have? "))
    input_skills = pd.Series()
    for i in range(amount_skills):
        temp_skill = input("Name skill #" + str(i + 1) + " ")
        input_skills = input_skills.append(pd.Series(temp_skill))

Enter your profession, or enter x to use default inputs x
Default profession:  Data Analyst
Default skills:  ['Python', 'R', 'SQL', 'Tableau', 'Power BI', 'Salesforce', 'Excel', 'Machine Learning', 'Statistics', 'Analytics', 'Visualization', 'Web Scraping']


In [5]:
input_skills_string = " ".join(input_skills) + " " + input_profession

In [6]:
# Import TfidfVectorizer
jobs_df["words"] = jobs_df["title"] + jobs_df["summary"] + jobs_df["description"]

# Create a TfidfVectorizer: tfidf
tfidf = TfidfVectorizer(stop_words="english", 
                        analyzer='word',
                        vocabulary = map(lambda x: str(x).lower(), skills_df.name.values),
                        ngram_range=(1, 3), max_df=0.1, min_df=0.0025, max_features=None)

# Apply fit_transform to job descriptions: tfidf_jobs
job_summaries = jobs_df["words"].astype(str).copy()
job_summaries = job_summaries.append(pd.Series([input_skills_string]))  

tfidf_result = tfidf.fit_transform(job_summaries.values)

tfidf_jobs = pd.DataFrame(data = tfidf_result.toarray(), 
                          index = job_summaries, 
                          columns = tfidf.get_feature_names())

tfidf_jobs.tail()

(isc)2  .net  .net clr  \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...     0.0   0.0       0.0   
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...     0.0   0.0       0.0   
Full Time Opportunities for PhD Students or Rec...     0.0   0.0       0.0   
Full Time Opportunities for PhD Students or Rec...     0.0   0.0       0.0   
Python R SQL Tableau Power BI Salesforce Excel ...     0.0   0.0       0.0   

                                                    .net compact framework  \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...                     0.0   
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...                     0.0   
Full Time Opportunities for PhD Students or Rec...                     0.0   
Full Time Opportunities for PhD Students or Rec...                     0.0   
Python R SQL Tableau Power BI Salesforce Excel ...                     0.0   

                                                    .net framework  \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...             0.0   
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...             0.0   
Full Time Opportunities for PhD Students or Rec...             0.0   
Full Time Opportunities for PhD Students or Rec...             0.0   
Python R SQL Tableau Power BI Salesforce Excel ...             0.0   

                                                    .net remoting  .net core  \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...            0.0        0.0   
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...            0.0        0.0   
Full Time Opportunities for PhD Students or Rec...            0.0        0.0   
Full Time Opportunities for PhD Students or Rec...            0.0        0.0   
Python R SQL Tableau Power BI Salesforce Excel ...            0.0        0.0   

                                                    .com  .htaccess  \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...   0.0        0.0   
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...   0.0        0.0   
Full Time Opportunities for PhD Students or Rec...   0.0        0.0   
Full Time Opportunities for PhD Students or Rec...   0.0        0.0   
Python R SQL Tableau Power BI Salesforce Excel ...   0.0        0.0   

                                                    1-4 units  ...  zumba  \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...        0.0  ...    0.0   
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...        0.0  ...    0.0   
Full Time Opportunities for PhD Students or Rec...        0.0  ...    0.0   
Full Time Opportunities for PhD Students or Rec...        0.0  ...    0.0   
Python R SQL Tableau Power BI Salesforce Excel ...        0.0  ...    0.0   

                                                    zumba instruction  zuora  \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...                0.0    0.0   
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...                0.0    0.0   
Full Time Opportunities for PhD Students or Rec...                0.0    0.0   
Full Time Opportunities for PhD Students or Rec...                0.0    0.0   
Python R SQL Tableau Power BI Salesforce Excel ...                0.0    0.0   

                                                    zymography  zynx  zyxel  \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...         0.0   0.0    0.0   
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...         0.0   0.0    0.0   
Full Time Opportunities for PhD Students or Rec...         0.0   0.0    0.0   
Full Time Opportunities for PhD Students or Rec...         0.0   0.0    0.0   
Python R SQL Tableau Power BI Salesforce Excel ...         0.0   0.0    0.0   

                                                    z/os  z/vm  zlinux  \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...   0.0   0.0     0.0   
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda...   0.0   0.0     0.0   
Full Time Opportunities for PhD Students or Rec...   0.0   0.0     0.0   
Full Time Opportunities for PhD Students or Rec...   0.0   0.0     0.0

In [7]:
def get_recommendations(features, index):

    #cosine similarity
    norm_features = normalize(features.toarray())
    jobs_rec_df = pd.DataFrame(norm_features, index = index, columns = tfidf.get_feature_names())
    current_item = jobs_rec_df.loc[input_skills_string].values.reshape(-1,1)
    similarities = jobs_rec_df.dot(current_item).reset_index()
    recommended_jobs = similarities.nlargest(n=6, columns=0)
    recommended_jobs = pd.merge(recommended_jobs.reset_index(), 
                                jobs_df, 
                                left_on="index", 
                                right_on="words")[["title","company","summary","link"]]
    return jobs_rec_df, similarities, recommended_jobs

In [8]:
jobs_rec_df, similarities, recommended_jobs = get_recommendations(tfidf_result, job_summaries)

In [9]:
similarities.sort_values(0, ascending = False).head()

,index,0
523,Python R SQL Tableau Power BI Salesforce Excel...,1.000000
395,Data Scientist Projects will includ...,0.255816
495,Junior Data Scientist PMC is buildi...,0.233016
402,"Data Scientist Medscape, a division...",0.219162
399,Data Scientist Reframe objectives a...,0.214770


In [10]:
for i in range(5):
    print(recommended_jobs["title"][i], "at", (recommended_jobs["company"][i]))
    print(recommended_jobs["summary"][i])
    print(recommended_jobs["link"][i])
    print()
    print()

Data Scientist at Hire Tech Solutions
            Projects will include creating machine learning microservices, detecting patterns and anomalies, analysis and interpretation of data, and data visualization....
https://www.indeed.com/company/Hire-Tech-Solutions/jobs/Data-Scientist-466f6ae6ab72506d?fccid=821241ecf9db90ef&vjs=3


Junior Data Scientist at Protect My Car
            PMC is building out a small team of talented professionals with demonstrated ability to conduct rigorous statistical analysis and apply machine learning...
https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Cc2767xsnTKh3D1ndxis86rSe8bm3rZJt-h0l22W5JsqVSPirPG7nXDv4eNXyw1s-UDmSqiHCKjDOqRXiD1rrvXb4iCZWBN71e31Fw4mZ3DhiDh6cuu3P8OlrrAbBcCk43ZmgQqEUjLSkWyaQeoONv6A02UjczxoGdAK7Cy2znFbM207UUMdF-PM2-61lKHybnVtw1UNU59-d3ylwFHoI_u1DG2mPMLCsraHoTj-kEac5S5_LLjdaRV8bRP4bW-N9sdj1SyQNRO0HfP2-FXLQQ8IOEhqGADC3Pwivr4iLfS93XVamqethjih-W1TxftLvn2lKue2HhrNIxH_WHGnxicSmSYz161pvSu9-P0pZo6UH_9FzesjusptNat7u2SEzf6JMumTpxXGMsa_tYwQEaQDXX

In [12]:
skills_rec = cosine_similarity(jobs_rec_df.transpose(), jobs_rec_df.transpose())
skills_rec_df = pd.DataFrame(skills_rec, index = list(skills_df.name.values), columns = list(skills_df.name.values))
pd.to_gbq(Projects.skills_matrix, project_id = project_id, if_exist = "replace")
skills_rec_df.head()

AttributeError: module 'pandas' has no attribute 'to_gbq'

In [ ]:
def skill_recommender(skill_list, matrix):
    skill_recs_all = pd.Series()
    skill_vals_all = pd.Series()  
    for skill in skill_list:
        result = matrix.sort_values(skill,ascending=False)
        skill_recs = pd.Series()
        skill_vals = pd.Series()
        for i in range(1,5): 
            temp_skill_recs = result[skill].index.values[i]
            temp_skill_val = str(round(result[skill][i],3))
            skill_recs = skill_recs.append(pd.Series(temp_skill_recs))
            skill_vals = skill_vals.append(pd.Series(temp_skill_val))
        skill_recs_all = skill_recs_all.append(skill_recs)
        skill_vals_all = skill_vals_all.append(skill_vals)
    skill_recommendations = pd.concat([skill_recs_all, skill_vals_all], axis=1, ignore_index=True)
    return skill_recommendations.drop_duplicates(subset='skill_recs_all')

In [ ]:
skill_recommender(input_skills, skills_rec_df)